In [ ]:
%%capture
! pip install transformers datasets evaluate accelerate>=0.26.0 transformers[torch]

In [48]:
import accelerate
print(accelerate.__version__)

1.4.0


In [2]:
import sys
print(sys.version)

3.11.9 (tags/v3.11.9:de54cf5, Apr  2 2024, 10:12:12) [MSC v.1938 64 bit (AMD64)]


In [3]:
%%capture
from datasets import load_dataset
imdb = load_dataset("imdb")

In [5]:
print(imdb["train"][0])
print(imdb["test"][0])
print(imdb["train"].shape, imdb["test"].shape)
print(imdb["train"])

{'text': 'I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967. I also heard that at first it was seized by U.S. customs if it ever tried to enter this country, therefore being a fan of films considered "controversial" I really had to see this for myself.<br /><br />The plot is centered around a young Swedish drama student named Lena who wants to learn everything she can about life. In particular she wants to focus her attentions to making some sort of documentary on what the average Swede thought about certain political issues such as the Vietnam War and race issues in the United States. In between asking politicians and ordinary denizens of Stockholm about their opinions on politics, she has sex with her drama teacher, classmates, and married men.<br /><br />What kills me about I AM CURIOUS-YELLOW is that 40 years ago, this was considered pornographic. Really, the sex and nudity scenes are few and far be

In [6]:
import pandas as pd
X_train = pd.DataFrame(imdb["train"]['text'][:25000]).rename(columns={0: 'text'})
y_train = pd.DataFrame(imdb["train"]['label'][:25000]).rename(columns={0: 'target'})
X_test = pd.DataFrame(imdb["test"]['text'][:25000]).rename(columns={0: 'text'})
y_test = pd.DataFrame(imdb["test"]['label'][:25000]).rename(columns={0: 'target'})
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)
del(imdb)

(25000, 1) (25000, 1) (25000, 1) (25000, 1)


In [7]:
n=1000
n_per_label =  (y_train['target'].value_counts()/len(y_train)* n).round().astype(int)
indices_sample = []
for label, count in n_per_label.items():
    # Sélectionner les indices pour chaque label
    label_indices = y_train[y_train['target'] == label].sample(n=count, random_state=42).index
    indices_sample.extend(label_indices)

y_train_spl = y_train.iloc[indices_sample]
X_train_spl = X_train.iloc[indices_sample]

n=250
n_per_label =  (y_test['target'].value_counts()/len(y_test)* n).round().astype(int)
indices_sample = []
for label, count in n_per_label.items():
    # Sélectionner les indices pour chaque label
    label_indices = y_test[y_test['target'] == label].sample(n=count, random_state=42).index
    indices_sample.extend(label_indices)

y_test_spl = y_test.iloc[indices_sample]
X_test_spl = X_test.iloc[indices_sample]

print(y_train_spl.shape, X_train_spl.shape, y_test_spl.shape, X_test_spl.shape)

(1000, 1) (1000, 1) (250, 1) (250, 1)


In [53]:
from datasets import Dataset
df_train = {'text' : X_train_spl.text.tolist(), 'label': y_train_spl.target.tolist()}
df_test = {'text' : X_test_spl.text.tolist(), 'label': y_test_spl.target.tolist()}

df_train = pd.DataFrame(df_train)
df_test = pd.DataFrame(df_test)

ds_train = Dataset.from_pandas(df_train)
ds_test = Dataset.from_pandas(df_test)

import pickle

with open('ds_train.pkl', 'wb') as f:
    pickle.dump(ds_train, f)

with open('ds_test.pkl', 'wb') as f:
    pickle.dump(ds_test, f)

In [10]:
%%capture
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
def preprocess_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)
from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [11]:
%%capture
tokenized_train = ds_train.map(preprocess_function, batched=True)
tokenized_test = ds_test.map(preprocess_function, batched=True)

In [16]:
tokenized_train

Dataset({
    features: ['text', 'label', 'input_ids', 'attention_mask'],
    num_rows: 1000
})

In [19]:
len(tokenized_train['input_ids'][0])

512

In [ ]:
%%capture
import evaluate
accuracy = evaluate.load("accuracy")

import numpy as np
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

# def compute_metrics(p):
#     predictions, labels = p
#     predictions = torch.argmax(torch.tensor(predictions), axis=1)
#     accuracy = (predictions == labels).sum().item() / len(labels)
#     return {"accuracy": accuracy}

In [23]:
#! pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu126

import torch
print(torch.cuda.is_available())
!nvidia-smi

x = torch.rand(5, 3)
print(x)

True
Wed Mar  5 23:29:34 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.70                 Driver Version: 560.70         CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                  Driver-Model | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce GTX 1660 Ti   WDDM  |   00000000:01:00.0  On |                  N/A |
| N/A   51C    P8             13W /   80W |     356MiB /   6144MiB |     17%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+

In [28]:
%%capture
! pip install peft

In [31]:
id2label = {0: "NEGATIVE", 1: "POSITIVE"}
label2id = {"NEGATIVE": 0, "POSITIVE": 1}

from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

# "distilbert-base-uncased"  "huawei-noah/TinyBERT_General_4L_312D"  "albert-base-v2" "facebook/bart-base" "t5-small"
model_name= "distilbert-base-uncased"

model = AutoModelForSequenceClassification.from_pretrained(
    model_name, num_labels=2, id2label=id2label, label2id=label2id
)
total_params = sum(p.numel() for p in model.parameters())
print("Total number of parameters:", f"{total_params:,}".replace(",", " ") )

for name, module in model.named_modules():
    print(name)

Total number of parameters: 66 955 010

distilbert
distilbert.embeddings
distilbert.embeddings.word_embeddings
distilbert.embeddings.position_embeddings
distilbert.embeddings.LayerNorm
distilbert.embeddings.dropout
distilbert.transformer
distilbert.transformer.layer
distilbert.transformer.layer.0
distilbert.transformer.layer.0.attention
distilbert.transformer.layer.0.attention.dropout
distilbert.transformer.layer.0.attention.q_lin
distilbert.transformer.layer.0.attention.k_lin
distilbert.transformer.layer.0.attention.v_lin
distilbert.transformer.layer.0.attention.out_lin
distilbert.transformer.layer.0.sa_layer_norm
distilbert.transformer.layer.0.ffn
distilbert.transformer.layer.0.ffn.dropout
distilbert.transformer.layer.0.ffn.lin1
distilbert.transformer.layer.0.ffn.lin2
distilbert.transformer.layer.0.ffn.activation
distilbert.transformer.layer.0.output_layer_norm
distilbert.transformer.layer.1
distilbert.transformer.layer.1.attention
distilbert.transformer.layer.1.attention.dropout
dis

In [33]:
from peft import LoraConfig, TaskType

lora_config = LoraConfig(
    task_type=TaskType.SEQ_CLS, r=1, lora_alpha=1, lora_dropout=0.1,
    target_modules=["q_lin", "k_lin", "v_lin"]
)

from peft import get_peft_model
model = get_peft_model(model, lora_config)

In [40]:
# Compter les paramètres entraînables
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

# Compter tous les paramètres du modèle
total_params = sum(p.numel() for p in model.parameters())

# Calculer le pourcentage de paramètres entraînables
trainable_percentage = (trainable_params / total_params) * 100

def format_with_spaces(number):
    return "{:,}".format(number).replace(",", " ")
print(f"Trainable params: {format_with_spaces(trainable_params)}")
print(f"Total params: {format_with_spaces(total_params)}")
print(f"Trainable%: {trainable_percentage:.6f}")

Trainable params: 619 778
Total params: 67 574 788
Trainable%: 0.917173


In [24]:
#torch.cuda.empty_cache()
#!nvidia-smi --gpu-reset -i 0
#!kill -9 $(pgrep -f 'python')

In [51]:
import transformers

In [52]:
%%time
from transformers import AdamW, get_scheduler, Adafactor # LAMB,
from torch.optim import AdamW

# Commence avec un batch size modéré (8 ou 16) et surveille la mémoire GPU avec nvidia-smi.
# Si l'utilisation mémoire GPU est faible, augmente la taille du batch pour tirer parti du GPU, mais reste dans les limites de la mémoire.
# Si l'entraînement devient trop lent, réduis la taille du batch et teste l'effet sur les performances.
# Si la mémoire est saturée, utilise gradient_accumulation_steps pour simuler un plus grand batch tout en utilisant moins de mémoire.
torch.cuda.empty_cache() # pour vider le GPU

training_args = TrainingArguments(
    output_dir="my_awesome_model",
    learning_rate=2e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    gradient_accumulation_steps=16,
    num_train_epochs=5,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=False,
    report_to='none',
    fp16=True,
    logging_dir='./logs',
    logging_steps=1,
    logging_first_step=True,
    save_steps=100
)

# Créer un optimiseur AdamW personnalisé
# optimizer = AdamW(model.parameters(), lr=training_args.learning_rate, weight_decay=training_args.weight_decay)
# optimizer = LAMB(model.parameters(), lr=training_args.learning_rate, weight_decay=training_args.weight_decay)
optimizer = Adafactor(model.parameters(), lr=training_args.learning_rate, relative_step=False, weight_decay=training_args.weight_decay)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    optimizers=(optimizer, None),  # Pas de scheduler personnalisé ici
)

trainer.train()

c:\Users\quent\Desktop\Projets\env\py3119\Lib\site-packages\transformers\training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


ImportError: Using the `Trainer` with `PyTorch` requires `accelerate>=0.26.0`: Please run `pip install transformers[torch]` or `pip install 'accelerate>=0.26.0'`

In [ ]:
results = trainer.evaluate()
print(results)